In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id="notebook_id"></a>
# Hash functions

A hash function maps a key $k$ to a value between $0$ and $m-1$ where $m$ is the number of buckets (or the size of the array) in the hash table. A good hash function should distribute keys evenly among the $m$ buckets; in other words, the probability that a key is mapped to any given bucket should be $1 / m$.

No one hash function is guaranteed to work well for an arbitrary set of keys. In fact, a malicious attacker that has knowledge of the hash function can cause the performance of a hash table to degrade by deliberately using keys that cause collisions in the hash table (interested readers can refer to the articles [here](https://static.usenix.org/event/sec03/tech/full_papers/crosby/crosby_html/) and [here](https://emboss.github.io/blog/2012/12/14/breaking-murmur-hash-flooding-dos-reloaded/)).

## The division method

The division method uses the same technique that we used in the previous notebook: The hash function computes the remainder after dividing the key value $k$ by $m$:

$$h(k) = k\ \text{mod}\ m$$

$\text{mod}$ is the modulo operator. For two positive integer values $a$ and $n$, $a\ \text{mod}\ n$ is the remainder after division of $a$ by $n$ using integer math. For example, $12 \ \text{mod}\ 4 = 0$ because $12$ divided by $4$ equals $3$ with no remainder. $25 \ \text{mod}\ 4 = 1$ because $25$ divided by $4$ equals $6$ with a remainder of $1$. In Java, the `%` operator can be used to compute the remainder after division; however, in linear probing we want to use the remainder as an array index which means that the remainder must not be negative. Unfortunately, the sign of the value returned by `%` is equal to the sign of the dividend (or $a$):

In [ ]:
int a = 12;
int n = 4;
System.out.println(a % n);

a = 25;
n = 4;
System.out.println(a % n);

a = -25;
n = 4;
System.out.println(a % n);

Because the number of buckets $m$ is always positive, we can safely use `Math.floorMod(k, m)` to compute the remainder if the key value $k$ can take on negative values.

Computing the remainder after division requires one division operation which makes the hash function quite fast.

Certain values of $m$ are generally avoided when using the division method. In particular, values of $m = 2^n$ that are equal to integer powers of $2$ should not be used because division by $2^n$ in binary simply shifts the bits of the number $n$ places. For example, the following table illustrates dividing $46$ by $2$, $4$, $8$, and $16$.

| k | m | k / m  | remainder  |
|-: | -: | -: | -: |
|101110 = 46 | 10 = 2 | 10111 = 23 | 0 = 0 |
|101110 = 46 | 100 =4 | 1011 = 11 | 10 = 2 |
|101110 = 46 | 1000 = 8 | 101 = 5 | 110 = 6 |
|101110 = 46 | 10000 = 16 | 10 = 2 | 1110 = 14 |

Notice that the remainder of dividing $k$ by $m = 2^n$ is simply the $n$ right-most bits of $k$. This means that the hash value depends only on $n$ bits of $k$ instead of depending on all of the bits of $k$. To see why this happens, recall that $k / m$ is simply the number of times that $m$ can be subtracted from $k$. Consider $46 / 16$ in binary:

```
 101110
- 10000
-------
  11110 = 30 which is greater than 16 so subtract again
  
  11110
- 10000
-------
   1110 = 14 which is less than 16 so done; we subtracted twice so the quotient is equal to 2
```

All of those trailing zeros in the binary representation of $16$ causes the last 4 bits of $k$ to remain untouched when we perform the repeated subtractions!

What we want is for the binary representation of $m$ to be made up of roughly equal numbers of 1s and 0s. One way to achieve this is to require $m$ to be not too close to a power of $2$.

It is also often recommended that $m$ be a prime number. In the situation where keys are uniformly random, then whether $m$ is prime or not (or even if $m$ is a power of $2$) does not matter. In cases where the keys have some relation to one another, making $m$ prime can reduce the number of collisions. Consider the following simple simulation that is run for 100000 trials. In each trial, we simulate the division method by computing hash values for hash tables of length $m = 24$ (non-prime with many factors) and $m = 23$ (prime). We generate twelve keys having values between $1$ and $600$ where six of the keys are random but intentionally related to one another by a common factor $f$ and the other six keys values have random values (so some might also have a common factor $f$). We count the number of collisions by counting the number of unique hash values and subtracting that value from 12.

Run the following cell to see the results of the simulation.

In [ ]:
import java.util.ArrayList;
import java.util.HashSet;
import java.util.Random;

int f = 12;
Random rng = new Random();
int col = 0;         // number of collisions
int colPrime = 0;    // number of collisions
for (int trials = 0; trials < 100000; trials++) {
    ArrayList<Integer> keys = new ArrayList<>();
    for (int i = 0; i < 6; i++) {
        keys.add((rng.nextInt(100) + 1) * f);   // half the keys are multiples of f
        keys.add((rng.nextInt(600) + 1));
    }
    // hash the keys using m = 24
    ArrayList<Integer> hash = new ArrayList<>();
    for (Integer key : keys) {
        hash.add(key % 24);
    }
    HashSet<Integer> uniq = new HashSet<>(hash);
    col += 12 - uniq.size();
    
    // hash the keys using m = 23
    hash = new ArrayList<>();
    for (Integer key : keys) {
        hash.add(key % 23);
    }
    uniq = new HashSet<>(hash);
    colPrime += 12 - uniq.size();
}
System.out.println("collisions for m = 24 : " + col);
System.out.println("collisions for m = 23 : " + colPrime);

If the primeness of $m$ has no effect on the number of collisions, then we would expect the smaller hash table to have more collisions because there are fewer buckets. The simulation shows that this is the clearly not the case. When $f=12$, $m=12$ causes double the number of collisions than $m=23$.

**Exercise 1** Run the simulation for the following values of $f = 1, 5, 7, 11, 13, 17, 19$. Can you explain the results?

**Exercise 2** $f=12$ produces the second most number of collisions but any value of $f$ that shares common factors with $24$ will also produce more collisions than $f=23$. What values of $f$ produce the most collisions?

## The multiplication method

The multiplication method produces hash values that do not depend significantly on the value of $m$. The multiplication method has three steps:

1. multiply the key $k$ by some value $A$ where $0 < A < 1$
2. discard the integer part of $kA$ keeping only the fractional part
3. multiply the fractional part by $m$ and take the floor of the result (round the result down to the nearest integer)

As described above, the multiplication method uses floating-point arithmetic but fixed-point versions exist where all operations can be performed using integer multiplication and bit-level operations.

The purpose of Step 1 is to "randomize" the bits of the key value $k$. This requires choosing a value of $A$ that has some "random" mix of 1s and 0s (so choosing $A = 1 / 2^n$ for some integer value $n$ is a terrible choice). [Donald Knuth](https://en.wikipedia.org/wiki/Donald_Knuth) suggests using the value 

$$A = 1 - \phi = \frac{\sqrt{5} - 1}{2} \approx 0.618033988749895...$$ 

where $\phi$ is the [golden ratio](https://en.wikipedia.org/wiki/Golden_ratio).

The purpose of Steps 2 and 3 are to ensure the hash value lies in the range $0$ to $m-1$.

A worked example of using the multiplication method to compute a hash value is shown below for a key value $k = 235$, $m = 512$, and $A = \frac{\sqrt{5} - 1}{2}$:

**Step 1**
$$\begin{align}
h(k) & = 235 \times 0.618033988749895... \\
& = 145.237987356225
\end{align}
$$

**Step 2**
$$\begin{align}
h(k) & = h(k) - \text{fix}(h(k))\\
& = 145.237987356225... - 145\\
& = 0.237987356225...
\end{align}
$$

**Step 3**
$$\begin{align}
h(k) & = \text{floor}(0.237987356225... \times 512)\\
& = \text{floor}(121.849526387348...) \\
& = 121
\end{align}
$$

In Step 2, the function $\text{fix}$ rounds towards zero. $\text{fix}$ is used instead of $\text{floor}$ in case the key $k$ is negative.

## Hash functions for non-integer keys

So far we have assumed that keys are integer values but most objects that a programmer would want to store in a hash table are not integer values.

For the Java standard library hashed containers, hash values are 32-bit integer values. A hash value for any primitive type having 32 or fewer bits can be obtained by interpreting the bits as though it were an `int` value; in Java such types include `boolean`, `byte`, `char`, `short`, `int`, and `float`. A hash value for the 64-bit primitive types `double` and `long` can be obtained by splitting their 64-bit representation into two 32-bit values and then combining the two values in some bitwise fashion; details can be found by reading the documentation for [Double.hashCode](https://docs.oracle.com/javase/8/docs/api/java/lang/Double.html#hashCode--) and [Long.hashCode](https://docs.oracle.com/javase/8/docs/api/java/lang/Long.html#hashCode--).

For reference types (class types), Java uses the `hashCode` method to generate a 32-bit signed integer hash value. The `Object` version of `hashCode` has the following in its documentation:

> As much as is reasonably practical, the hashCode method defined by class Object does return distinct integers for distinct objects. (This is typically implemented by converting the internal address of the object into an integer, but this implementation technique is not required by the JavaTM programming language.)

When creating a new class where objects can be compared for equality (for example, two strings can be compared for equality by testing if they have the same length and the the same characters in the same order), the programmer overrides both the `equals` and `hashCode` methods. The responsibility of producing a good hash function falls on the programmer of the class; happily, most modern IDEs will generate a reasonable `hashCode` method automatically. The method used by the eclipse IDE is similar to the step-by-step recipe Joshua Bloch published for implementing a usable `hashCode` method in the book *Effective Java, Third Edition*. The recipe for overriding `hashCode` is quoted directly from its source below (but you don't need to know this recipe for CISC235).

How do the Java hashed container classes uses `hashCode` internally? The source code for `HashMap` tells us:

```java
   /**
     * Computes key.hashCode() and spreads (XORs) higher bits of hash
     * to lower.  Because the table uses power-of-two masking, sets of
     * hashes that vary only in bits above the current mask will
     * always collide. (Among known examples are sets of Float keys
     * holding consecutive whole numbers in small tables.)  So we
     * apply a transform that spreads the impact of higher bits
     * downward. There is a tradeoff between speed, utility, and
     * quality of bit-spreading. Because many common sets of hashes
     * are already reasonably distributed (so don't benefit from
     * spreading), and because we use trees to handle large sets of
     * collisions in bins, we just XOR some shifted bits in the
     * cheapest possible way to reduce systematic lossage, as well as
     * to incorporate impact of the highest bits that would otherwise
     * never be used in index calculations because of table bounds.
     */
    static final int hash(Object key) {
        int h;
        return (key == null) ? 0 : (h = key.hashCode()) ^ (h >>> 16);
    }
```


### Implementing `hashCode`

1. Declare an `int` variable named `result`, and initialize it to the hash code `c` for the first significant field in your object, as computed in Step 2A (Recall that a significant field is a field that affects equals comparisons.)
2. For every remaining significant field `f` in your object, do the following:
    1. Compute an `int` hash code `c` for the field:
        1. If the field is a primitive type, compute *`Type`* `.hashCode(f)`, where *`Type`* is the boxed primitive class corresponding to `f`'s type.
        2. If the field is an object reference and this class's `equals` method compares the field by recursively invoking `equals`, recursively invoke `hashCode` on the field. If a more complex comparison is required in `equals`, compute a "canonical representation" for this field and invoke `hashCode` on the canonical representation. If the value of the field is `null`, use 0 (or some other constant, but 0 is traditional).
        3. If the field is an array, treat it as if each significant element were a separate field. That is, compute a hash code for each significant element by applying these rules recursively, and combine the values per step 2B. If the array has no significant elements, use a constant, preferably 0. If all elements are significant, use `Arrays.hashCode`.
    2. Combine the hash code `c` computed in step 2A into `result` as follows:
    ```java
    result = 31 * result + c;
    ```
3. Return `result`.

The recipe seems somewhat complicated but is actually easy to implement in practice (except for the "canonical representation" part of Step 2A(b)